In [1]:
import torch
import os
import json
import time
import pandas as pd
import h3
import matplotlib.pyplot as plt
from tqdm import tqdm
from datasets import load_dataset, Dataset
from transformers import AutoTokenizer, AutoModel, TrainingArguments, Trainer, DataCollatorWithPadding
from torch import nn
from torch.utils.data import DataLoader
from torch.nn.utils.rnn import pad_sequence

print("This is three task")
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)
batch_size = 512
print("batch_size : ", batch_size)

address_data_path = f'/home/uddeshya.singh/DATA/PUNJAB_train/PUNJAB_with_grid_ids_train.csv'
fine_tuned_lm_trained_save_dir = '/home/uddeshya.singh/level-two_models/Fine-tuned-save'
fine_tuned_lm_trained_out_dir = '/home/uddeshya.singh/level-two_models/Fine-tuned-out'
model_dir = '/home/uddeshya.singh/Models/pan_india_train_threshold_save_dir/'

pretrained_tokenizer = AutoTokenizer.from_pretrained(model_dir)

print('Starting now ', time.strftime('%X %x %Z'))
df_train = pd.read_csv(address_data_path)
print('Completed loading ', time.strftime('%X %x %Z'))

def compute_grid_ids(row, res):
    return h3.geo_to_h3(row['lookup_lat'], row['lookup_lng'], res)

df_train['grid_id_res8'] = df_train.apply(compute_grid_ids, axis=1, res=8)
df_train['grid_id_res9'] = df_train.apply(compute_grid_ids, axis=1, res=9)
df_train['grid_id_res10'] = df_train.apply(compute_grid_ids, axis=1, res=10)

address_data_train = Dataset.from_pandas(df_train)
print(address_data_train)

def tokenize(batch):
    return pretrained_tokenizer(batch["cleaned_address"], padding=True, truncation=True, max_length=100)

def clean_dataset(batch):
    if pd.isnull(batch['cleaned_address']) or batch['cleaned_address'] == '':
        return False
    return True

print('starting now ', time.strftime('%X %x %Z'))
address_data_train = address_data_train.filter(clean_dataset)
print('completed now ', time.strftime('%X %x %Z'))

address_data_train = address_data_train.remove_columns(['addr_hash', 'lookup_lat', 'lookup_lng', 'grid_id'])
address_data_train = address_data_train.class_encode_column("grid_id_res8")
address_data_train = address_data_train.class_encode_column("grid_id_res10")
address_data_train = address_data_train.class_encode_column("grid_id_res9")


num_labels_res8 = address_data_train.features["grid_id_res8"].num_classes
num_labels_res10 = address_data_train.features["grid_id_res10"].num_classes
num_labels_res9 = address_data_train.features["grid_id_res9"].num_classes

config = {
    "num_labels_res8": num_labels_res8,
    "num_labels_res10": num_labels_res10,
    "num_labels_res9": num_labels_res9
}
with open(os.path.join(fine_tuned_lm_trained_save_dir, 'config.json'), 'w') as f:
    json.dump(config, f)



This is three task
cuda
batch_size :  512
Starting now  18:17:08 08/01/24 IST
Completed loading  18:17:21 08/01/24 IST
Dataset({
    features: ['addr_hash', 'cleaned_address', 'lookup_lat', 'lookup_lng', 'grid_id', 'grid_id_res8', 'grid_id_res9', 'grid_id_res10'],
    num_rows: 6913138
})
starting now  18:21:43 08/01/24 IST


  0%|          | 0/6914 [00:00<?, ?ba/s]

completed now  18:23:01 08/01/24 IST


Flattening the indices:   0%|          | 0/6914 [00:00<?, ?ba/s]

Casting to class labels:   0%|          | 0/6914 [00:00<?, ?ba/s]

Casting the dataset:   0%|          | 0/692 [00:00<?, ?ba/s]

Casting to class labels:   0%|          | 0/6914 [00:00<?, ?ba/s]

Casting the dataset:   0%|          | 0/692 [00:00<?, ?ba/s]

Casting to class labels:   0%|          | 0/6914 [00:00<?, ?ba/s]

Casting the dataset:   0%|          | 0/692 [00:00<?, ?ba/s]